In [20]:
!pip install numpy pandas scikit-learn tensorflow imblearn openpyxl

In [21]:
#Importation des librairies communes (peu prendre du temps)
import os
import sys
import numpy as np
import pandas as pd
#Chemin menant aux données
data_path = os.path.join(os.getcwd(), "source")
#création du dictionnaire
data_dict = dict()
# Boucle pour récupérer tous les fichiers CSV dans le dossier et les ajouter au dictionnaire
for file in os.listdir(data_path):
    if file.endswith(".csv"):
        file_path = os.path.join(data_path, file)
        dict_key = file.split('.')[0]  # Nom du fichier sans l'extension .csv
        data_dict[dict_key] = pd.read_csv(file_path)
data_dict.keys()

dict_keys(['appearances', 'club_games', 'clubs', 'competitions', 'game_events', 'game_lineups', 'games', 'player_valuations', 'players', 'clubs_link_api'])

In [23]:
#Toutes les fonctions
def games_of_the_season(df_games, club_id, saison, lieu='tous'):
    """
    Récupère tous les matchs d'une saison pour un club spécifié, ou les 10 derniers matchs si moins de 5 matchs dans la saison.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    saison (str) : Saison à filtrer.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Filtrage initial par saison et lieu
    if lieu == 'domicile':
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) & (df_games['season'] == saison)]
    elif lieu == 'exterieur':
        matchs_du_club = df_games[(df_games['away_club_id'] == club_id) & (df_games['season'] == saison)]
    else:
        matchs_du_club = df_games[((df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)) & (df_games['season'] == saison)]
    
    # Vérification du nombre de matchs
    if len(matchs_du_club) < 5:
        # Prendre les 10 derniers matchs toutes saisons confondues si moins de 5 matchs dans la saison
        matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(10)
    else:
        # Trier par date
        matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'season', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]

def last_five_games(df_games, club_id, date, lieu='tous'):
    """
    Récupère les 5 derniers matchs d'un club spécifié avant une date donnée en fonction du lieu du match.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    date (str ou pd.Timestamp) : Date avant laquelle les matchs doivent être récupérés.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    DataFrame : Les matchs demandés avec les informations détaillées.
    """
    # Assurer que la colonne 'date' est au format datetime
    df_games['date'] = pd.to_datetime(df_games['date'])

    # Filtrage des matchs avant la date donnée
    matchs_du_club = df_games[df_games['date'] < date]
    
    if lieu == 'domicile':
        matchs_du_club = matchs_du_club[matchs_du_club['home_club_id'] == club_id]
    elif lieu == 'exterieur':
        matchs_du_club = matchs_du_club[matchs_du_club['away_club_id'] == club_id]
    else:  # 'tous'
        matchs_du_club = matchs_du_club[(matchs_du_club['home_club_id'] == club_id) | (matchs_du_club['away_club_id'] == club_id)]
    
    # Tri des matchs par date et sélection des 5 derniers
    matchs_du_club = matchs_du_club.sort_values(by='date', ascending=False).head(5)

    # Ajout des informations supplémentaires
    matchs_du_club['adversaire_id'] = matchs_du_club.apply(lambda row: row['away_club_id'] if row['home_club_id'] == club_id else row['home_club_id'], axis=1)
    matchs_du_club['lieu'] = matchs_du_club.apply(lambda row: 'domicile' if row['home_club_id'] == club_id else 'exterieur', axis=1)
    matchs_du_club['résultat'] = matchs_du_club.apply(lambda row: 'victoire' if (row['home_club_id'] == club_id and row['home_club_goals'] > row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] > row['home_club_goals']) else 'défaite' if (row['home_club_id'] == club_id and row['home_club_goals'] < row['away_club_goals']) or (row['away_club_id'] == club_id and row['away_club_goals'] < row['home_club_goals']) else 'nul', axis=1)

    return matchs_du_club[['date', 'home_club_id', 'away_club_id', 'adversaire_id', 'lieu', 'home_club_goals', 'away_club_goals', 'résultat']]
    
def last_game_id(df_games, club_id):
    """
    Récupère l'ID du dernier match joué par un club spécifié.

    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.

    Retourne:
    int : ID du dernier match joué par le club.
    """
    # Filtrer les matchs impliquant le club
    matchs_du_club = df_games[(df_games['home_club_id'] == club_id) | (df_games['away_club_id'] == club_id)]
    
    # Trier par date en ordre décroissant et prendre le premier match
    dernier_match = matchs_du_club.sort_values(by='date', ascending=False).iloc[0]

    return dernier_match['game_id']

def match_sheet(df_game_lineups, match_id, club_id):
    """
    Récupère tous les player_id qui ont participé à un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs qui ont participé au match.
    """
    # Filtrer pour obtenir les lignes correspondant au match_id et club_id spécifiés
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & (df_game_lineups['club_id'] == club_id)]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

def starting_lineup_match(df_game_lineups, match_id, club_id):
    """
    Récupère les player_id des joueurs qui étaient dans la composition de départ d'un match spécifique pour un club donné.
    
    Paramètres:
    df_game_lineups (DataFrame) : DataFrame des compositions d'équipe.
    match_id (int) : ID du match.
    club_id (int) : ID du club.
    
    Retourne:
    Liste : Les identifiants (player_id) des joueurs de la composition de départ.
    """
    #df_game_lineups = data_dict['game_lineups']
    # Filtrer pour obtenir les lignes correspondant au match_id, club_id et type 'starting_lineup'
    lineup_du_match = df_game_lineups[(df_game_lineups['game_id'] == match_id) & 
                                      (df_game_lineups['club_id'] == club_id) & 
                                      (df_game_lineups['type'] == 'starting_lineup')]
    
    # Récupérer les player_id
    joueurs = lineup_du_match['player_id'].tolist()

    return joueurs

def market_value_player(df_player, player_id):
    """
    Récupère la valeur marchande d'un joueur spécifié.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    player_id (int) : ID du joueur.
    
    Retourne:
    float : La valeur marchande du joueur en euros.
    """
    # Trouver l'entrée correspondant au player_id
    valeur_joueur = df_player[df_player['player_id'] == player_id]['market_value_in_eur']

    # Retourner la valeur marchande, ou None si le joueur n'est pas trouvé
    return valeur_joueur.iloc[0] if not valeur_joueur.empty else None

def mean_team_value(df_player, liste_player_id):
    """
    Calcule la valeur marchande moyenne d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Valeur marchande moyenne de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs) / len(valeurs) if valeurs else 0

def sum_team_value(df_player, liste_player_id):
    """
    Calcule la somme des valeurs marchandes d'une équipe.
    
    Paramètres:
    df_player_valuations (DataFrame) : DataFrame des évaluations des joueurs.
    liste_player_id (List[int]) : Liste des ID des joueurs.
    
    Retourne:
    float : Somme des valeurs marchandes de l'équipe.
    """
    valeurs = [market_value_player(df_player, player_id) for player_id in liste_player_id]
    valeurs = [v for v in valeurs if v is not None]  # Exclure les valeurs None
    return sum(valeurs)

def predire_resultat(model, match_features, scaler, seuil_nul=0.5):
    """
    Prédit le résultat d'un match de football en utilisant un modèle de réseau de neurones entraîné.
    
    Paramètres :
    model : Le modèle de réseau de neurones entraîné.
    match_features : Un DataFrame contenant les caractéristiques du match.
    scaler : L'objet StandardScaler utilisé pour normaliser les caractéristiques d'entraînement.
    seuil_nul : Seuil de confiance pour prédire un match nul.
    
    Retourne :
    str : La prédiction du résultat du match ('victoire', 'nul', 'défaite').
    """
    # Normalisation des caractéristiques du match
    match_features_scaled = scaler.transform(match_features)

    # Faire la prédiction
    prediction_prob = model.predict(match_features_scaled)
    
    # Obtenir l'indice de la classe avec la probabilité la plus élevée
    predicted_class = np.argmax(prediction_prob, axis=1)[0]

    # Ajuster la prédiction en fonction du seuil pour les matchs nuls
    if prediction_prob[0][1] >= seuil_nul:  # Supposons que l'indice 1 représente les matchs nuls
        resultat = "nul"
    else:
        resultat = "victoire" if predicted_class == 0 else "defaite"  # Supposons que l'indice 2 représente les victoires

    return resultat


def get_name(club_id):
    """
    Renvoie le nom du club de football basé sur son identifiant.

    Paramètres:
    club_id (int): L'identifiant du club.

    Retourne:
    str: Le nom du club.
    """
    # Trouver la ligne correspondante au club_id
    club_info = df_clubs[df_clubs['club_id'] == club_id]

    # Renvoie le nom du club s'il est trouvé, sinon renvoie None
    if not club_info.empty:
        return club_info['name'].iloc[0]
    else:
        return None

def calculer_forme(df_games, club_id, date_du_match, lieu='tous'):
    """
    Calcule la forme d'un club sur ses 5 derniers matchs avant la date du match spécifiée.
    
    Paramètres:
    df_games (DataFrame) : DataFrame des matchs.
    club_id (int) : ID du club.
    date_du_match (str ou pd.Timestamp) : Date du match pour lequel la forme est calculée.
    lieu (str) : 'domicile', 'exterieur', ou 'tous' pour filtrer les matchs.
    
    Retourne:
    int : Les points accumulés par le club sur ses 5 derniers matchs.
    """
    # Assurer que la colonne 'date' est au format datetime
    #df_games['date'] = pd.to_datetime(df_games['date'])

    # Filtrer les matchs précédents et trier par date
    matchs_precedents = last_five_games(df_games, club_id, date_du_match, lieu)
    
    # Calculer les points (exemple simplifié)
    points = 0
    for _, row in matchs_precedents.iterrows():
        if row['home_club_id'] == club_id:
            points += 3 if row['home_club_goals'] > row['away_club_goals'] else 1 if row['home_club_goals'] == row['away_club_goals'] else 0
        else:
            points += 3 if row['away_club_goals'] > row['home_club_goals'] else 1 if row['away_club_goals'] == row['home_club_goals'] else 0
    
    return points

In [24]:
import pandas as pd

df_final_path = "df_final.xlsx"
df_to_train_path = "df_to_train.xlsx"

# Charger le fichier Excel dans un DataFrame
df_final = pd.read_excel(df_final_path, engine='openpyxl')
df_final.head()

FileNotFoundError: [Errno 2] No such file or directory: 'df_final.xlsx'

In [ ]:
df_to_train = pd.read_excel(df_to_train_path, engine='openpyxl')
conditions = (
    (df_final['home_team_market_value'] >= 1) & 
    (df_final['away_team_market_value'] >= 1) & 
    (df_final['home_team_form'] >= 1) & 
    (df_final['away_team_form'] >= 1)
)
df_to_train = df_final[conditions]
df_to_train = df_to_train.drop(["season", "game_id", "away_club_position", "home_club_position", "date"], axis=1).dropna()

df_to_train.head()

In [5]:
print(f"Taille du df {len(df_to_train)}")
print("Nombre de lignes avec 0 dans 'resultat':", (df_to_train['resultat'] == 0).sum())
print("Nombre de lignes avec -1 dans 'resultat':", (df_to_train['resultat'] == -1).sum())
print("Nombre de lignes avec 1 dans 'resultat':", (df_to_train['resultat'] == 1).sum())
print("PB home market value:", (df_to_train['home_team_market_value'] < 1).sum())
print("PB away market value:", (df_to_train['away_team_market_value'] < 1).sum())
print("PB home market value:", (df_to_train['home_team_form'] < 1).sum())
print("PB away market value:",  (df_to_train['away_team_form'] < 1).sum())

Taille du df 33640
Nombre de lignes avec 0 dans 'resultat': 8349
Nombre de lignes avec -1 dans 'resultat': 10326
Nombre de lignes avec 1 dans 'resultat': 14965
PB home market value: 0
PB away market value: 0
PB home market value: 0
PB away market value: 0


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Supposons que df_final est déjà défini et contient les données nécessaires

# 1. Préparation et nettoyage des données
conditions = (
    (df_final['home_team_market_value'] >= 1) & 
    (df_final['away_team_market_value'] >= 1) & 
    (df_final['home_team_form'] >= 1) & 
    (df_final['away_team_form'] >= 1)
)
df_to_train = df_final[conditions]
df_to_train = df_to_train.drop(["season", "game_id", "away_club_position", "home_club_position", "date"], axis=1).dropna()

X = df_to_train.drop(["resultat"], axis=1)
y = df_to_train["resultat"]

# Pas de rééquilibrage des classes avec SMOTE ou UnderSampler ici

y_encoded = to_categorical(y, num_classes=3)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Normalisation des caractéristiques
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Construction et entraînement du modèle
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Calcul des poids des classes
# Supposons que dans votre encodage, la classe 0 correspond aux matchs nuls,
# 1 correspond aux défaites, et 2 correspond aux victoires à domicile

# Poids des classes personnalisés
# Augmentez ces valeurs pour augmenter l'importance de la victoire et de la défaite dans l'entraînement
poids_victoire = 2.25
poids_defaite = 2.0
poids_nul = 1.0  # Diminuez cette valeur pour diminuer l'importance des matchs nuls

# Créez un dictionnaire de poids
class_weights_custom = {
    0: poids_victoire,   # Match nul
    1: poids_nul,  # Défaite
    2: poids_defaite  # Victoire
}

# Entraînement avec les poids des classes
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train_scaled, 
    y_train_encoded, 
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    class_weight=class_weights_custom,  # Utilisation des poids des classes
    verbose=1
)

# 4. Évaluation du modèle
loss, accuracy = model.evaluate(X_test_scaled, y_test_encoded)
print(f'Loss: {loss}, Accuracy: {accuracy}')


Epoch 1/50
673/673 [==============================] - 3s 3ms/step - loss: 1.7574 - accuracy: 0.4221 - val_loss: 1.1331 - val_accuracy: 0.4195
Epoch 2/50
673/673 [==============================] - 2s 2ms/step - loss: 1.7104 - accuracy: 0.4434 - val_loss: 1.0776 - val_accuracy: 0.4458
Epoch 3/50
673/673 [==============================] - 2s 2ms/step - loss: 1.6932 - accuracy: 0.4448 - val_loss: 1.0910 - val_accuracy: 0.4525
Epoch 4/50
673/673 [==============================] - 2s 2ms/step - loss: 1.6839 - accuracy: 0.4435 - val_loss: 1.0798 - val_accuracy: 0.4380
Epoch 5/50
673/673 [==============================] - 1s 2ms/step - loss: 1.6778 - accuracy: 0.4434 - val_loss: 1.0946 - val_accuracy: 0.4442
Epoch 6/50
673/673 [==============================] - 2s 2ms/step - loss: 1.6756 - accuracy: 0.4474 - val_loss: 1.0691 - val_accuracy: 0.4341
Epoch 7/50
673/673 [==============================] - 2s 2ms/step - loss: 1.6725 - accuracy: 0.4470 - val_loss: 1.0683 - val_accuracy: 0.4442
Epoch 

In [7]:
"""
model = Sequential()
model.add(Dense(256, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
"""
"""
from keras.layers import Conv1D, MaxPooling1D, Flatten

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))
"""

"""
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(Dense(3, activation='softmax'))
"""

"\nfrom keras.layers import LSTM\n\nmodel = Sequential()\nmodel.add(LSTM(50, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))\nmodel.add(Dense(3, activation='softmax'))\n"

In [18]:
df_clubs = data_dict["clubs"]
df_players = data_dict['players']
df_games = data_dict['games']
df_game_lineups = data_dict['game_lineups']

# Liste des matchs pour lesquels faire des prédictions
#liste_matchs = [(1041, 583), (1041, 3524), (583, 3524), (1041, 162), (162, 1041), 
#                (418, 3302), (418, 12321), (31, 281), (281, 31), (418, 281), (281, 418), (281, 281)]
import random

# Liste des identifiants de club
club_ids = [
    162, 244, 273, 347, 415, 417, 583, 667, 738, 826,
    969, 995, 1041, 1082, 1158, 1421, 3524, 3911
]

# Générer des paires aléatoires de matchs
liste_matchs = []

while len(club_ids) > 1:  # Continue tant qu'il y a au moins 2 clubs pour former un match
    home = random.choice(club_ids)
    club_ids.remove(home)
    away = random.choice(club_ids)
    club_ids.remove(away)
    liste_matchs.append((home, away))

# Boucle sur chaque match pour faire des prédictions
for home_club_id, away_club_id in liste_matchs:
    match_features = pd.DataFrame({
        'home_club_id': [home_club_id],
        'away_club_id': [away_club_id],
        'home_team_market_value': [mean_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, home_club_id), home_club_id))],
        'away_team_market_value': [mean_team_value(df_players, starting_lineup_match(df_game_lineups, last_game_id(df_games, away_club_id), away_club_id))],
        'home_team_form': [calculer_forme(df_games, home_club_id, "2024-01-01", "domicile")],
        'away_team_form': [calculer_forme(df_games, away_club_id, "2024-01-01", "exterieur")]
    })

    # Faire la prédiction pour le match courant
    resultat_predit = predire_resultat(model, match_features, scaler, 0.4)
    print(f"Résultat prédit pour le match entre les clubs {get_name(home_club_id)} et {get_name(away_club_id)} : {resultat_predit}")


1/1 [==============================] - 0s 32ms/step
Résultat prédit pour le match entre les clubs Toulouse Football Club et Lille Olympique Sporting Club Lille Métropole : victoire
1/1 [==============================] - 0s 38ms/step
Résultat prédit pour le match entre les clubs Racing Club de Strasbourg Alsace et Paris Saint-Germain Football Club : defaite
1/1 [==============================] - 0s 32ms/step
Résultat prédit pour le match entre les clubs Stade de Reims et Association sportive de Monaco Football Club : victoire
1/1 [==============================] - 0s 26ms/step
Résultat prédit pour le match entre les clubs Football Club Lorient-Bretagne Sud et Olympique Lyonnais : victoire
1/1 [==============================] - 0s 32ms/step
Résultat prédit pour le match entre les clubs Olympique Gymnaste Club Nice Côte d'Azur et Racing Club de Lens : nul
1/1 [==============================] - 0s 32ms/step
Résultat prédit pour le match entre les clubs Football Club de Nantes et Olympique 

In [11]:
import joblib
# Supposons que `model` est votre modèle entraîné
joblib.dump(model, 'model_sequential.pkl')

['model_sequential.pkl']

In [12]:
import joblib
# Supposons que `model` est votre modèle entraîné
joblib.dump(scaler, 'scaler_sequential.pkl')

['scaler_sequential.pkl']